In [1]:
import numpy as np
import cv2
from PIL import Image
import math
import copy

In [2]:
sobel_X=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
sobel_Y=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])

image=Image.open('bicycle.bmp')
img_arr=np.array(image)

In [3]:
def rgb2gray(image):
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return gray

In [4]:
def gaussian_smoothing(sigma=2,mean=0):
    
    X=np.linspace(-3*sigma,3*sigma,3*int(sigma)+1)
    Y=np.linspace(-3*sigma,3*sigma,3*int(sigma)+1)
    [grid_X,grid_Y]=np.meshgrid(X,Y)
    
    gaussian_filter=np.exp(-(grid_X**2+grid_Y**2)/(2*sigma**2))/(math.sqrt(2*sigma)*sigma)
    
    return gaussian_filter

In [5]:
def convolution(I,H):
    
    H=np.rot90(np.rot90(H))
    
    if len(I.shape)==3:
        I=rgb2gray(I)

    (I_R,I_C)=I.shape
    (H_R,H_C)=H.shape
    
    conv=np.zeros(I.shape)
    
    for i in range(int(H_R/2),I_R-int(H_R/2)):
        for j in range(int(H_C/2),I_C-int(H_C/2)):
            window=I[i-H_R//2:i+H_R//2+1,j-H_C//2:j+H_C//2+1]
            conv[i,j]=np.sum(window*H)
    
    return conv
        

In [6]:
def harris_detector(I,alpha=0.04):
    
    I_gray=rgb2gray(I)
    gaussian_filter=gaussian_smoothing()
    
    Gaussian=cv2.GaussianBlur(I,(3,3),3)
    
    grad_X=convolution(Gaussian,sobel_X)
    grad_X=cv2.GaussianBlur(grad_X,(3,3),1.4)
    grad_Y=convolution(Gaussian,sobel_Y)
    grad_Y=cv2.GaussianBlur(grad_Y,(3,3),1.4)
    grad_XX=np.square(grad_X)
    grad_YY=np.square(grad_Y)
    grad_XY=np.multiply(grad_X,grad_Y)
    
    max_R=0
    mat={}
    for row in range(1,I.shape[0]-1):
        for clm in range(1,I.shape[1]-1):
            
            Window_XX=np.sum(grad_XX[row-1:row+2,clm-1:clm+2])
            Window_XY=np.sum(grad_XY[row-1:row+2,clm-1:clm+2])
            Window_YY=np.sum(grad_YY[row-1:row+2,clm-1:clm+2])
            
            M=np.array([[Window_XX,Window_XY],[Window_XY,Window_YY]])
            
            R=np.linalg.det(M)-alpha*(np.trace(M))**2
                        
            mat[(row,clm)]=R
            
            if R>max_R:
                max_R=R
    
    mat_copy=mat.copy()
    print(max_R)
    for k,v in mat_copy.items():
        
        if not v>max_R*0.05:
            del mat[k]
    
    points=[]
    for k in mat.keys():
        max_R=0
        for i in range(k[0]-1,k[0]+3):
            for j in range(k[1]-1,k[1]+3):
                
                if (i,j) in mat.keys():
                    
                    if mat[(i,j)]>max_R:
                        pt=(i,j)
                        max_R=mat[(i,j)]
        if pt not in points:                
            points.append(pt)
    
    for val in points:
        I=cv2.circle(I,(val[1],val[0]),1,(255,0,0))
    
#     for val in mat.keys():
#         I=cv2.circle(I,(val[1],val[0]),1,(255,0,0))
    
    image_try=Image.fromarray(I)
    image_try.show()
    
    return mat,grad_X,grad_Y

In [7]:
I,x,y=harris_detector(img_arr)

255458121849.48303


In [12]:
import matplotlib.pyplot as plt

In [21]:
image_try=Image.fromarray(y)
image_try.show()

In [22]:
image_try=Image.fromarray(x)
image_try.show()

In [38]:
x[-1:3,1:3]

array([], shape=(0, 2), dtype=float64)

In [59]:
image=Image.fromarray(cv2.circle(img_arr,(0,2000),40,(255,0,0),-1))
image.show()